<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto_Part 3</font></h1>

## Problem 3

- Explore and cluster the neighborhoods in Toronto.
- Generate maps to visualize your neighborhoods and how they cluster together

In [4]:
import pandas as pd

In [27]:
df = pd.read_csv('C:\\Users\\02484856\\Desktop\\IBM Projects\\Example Solution\\data_toronto.csv',header=None)
df.columns=["Postalcode","Borough","Neighbourhood","Latitude","Longitude"]
df.head(10)

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
0,Postalcode,Borough,Neighbourhood,Latitude,Longitude
1,M1B,Scarborough,"Rouge,Malvern",43.806686299999996,-79.19435340000001
2,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.7845351,-79.16049709999999
3,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.7635726,-79.1887115
4,M1G,Scarborough,Woburn,43.7709921,-79.21691740000001
5,M1H,Scarborough,Cedarbrae,43.773136,-79.23947609999999
6,M1J,Scarborough,Scarborough Village,43.7447342,-79.23947609999999
7,M1K,Scarborough,"Kennedy Park,Ionview,East Birchmount Park",43.7279292,-79.26202940000002
8,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711111700000004,-79.2845772
9,M1M,Scarborough,"Scarborough Village West,Cliffcrest,Cliffside",43.716316,-79.23947609999999


In [28]:
df.drop(df.index[0], inplace = True)

In [40]:
df['Latitude'] = pd.to_numeric(df['Latitude'],errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'],errors='coerce')
df.head()

,Postalcode,Borough,Neighbourhood,Latitude,Longitude
1,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
2,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497
3,M1E,Scarborough,"West Hill,Guildwood,Morningside",43.763573,-79.188711
4,M1G,Scarborough,Woburn,43.770992,-79.216917
5,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [36]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [41]:
# Get coordinates of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = "Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude

print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [42]:
# Derive map of Toronto
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label_f = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label_f,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [43]:
CLIENT_ID = 'V12WHXAVJQNMK5B1QGROV4M4ALZ4CVHTP1JHWRSSC3B2D3LO' # your Foursquare ID
CLIENT_SECRET = 'NATR34RL4O5UMDC03ZYQJKXCVXM5TLEB3NGJPCMHT4LZHRSH' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: V12WHXAVJQNMK5B1QGROV4M4ALZ4CVHTP1JHWRSSC3B2D3LO
CLIENT_SECRET:NATR34RL4O5UMDC03ZYQJKXCVXM5TLEB3NGJPCMHT4LZHRSH


In [47]:
import requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [51]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()
        items = results["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in items])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    # for venue_list in venues_list:
    #     for item in venue_list:
    #         item

    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [52]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])

Rouge,Malvern
Highland Creek,Port Union,Rouge Hill
West Hill,Guildwood,Morningside
Woburn
Cedarbrae
Scarborough Village
Kennedy Park,Ionview,East Birchmount Park
Clairlea,Golden Mile,Oakridge
Scarborough Village West,Cliffcrest,Cliffside
Birch Cliff,Cliffside West
Wexford Heights,Scarborough Town Centre,Dorset Park
Maryvale,Wexford
Agincourt
Clarks Corners,Tam O'Shanter,Sullivan
Steeles East,Milliken,Agincourt North,L'Amoreaux East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Henry Farm,Fairview,Oriole
Bayview Village
Silver Hills,York Mills
Willowdale,Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Wilson Heights,Downsview North
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill,Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
India Bazaar,The Beac

In [53]:
toronto_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge,Malvern",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Highland Creek,Port Union,Rouge Hill",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"West Hill,Guildwood,Morningside",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"West Hill,Guildwood,Morningside",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [54]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood,Long Branch",11,11,11,11,11,11
"Bathurst Manor,Wilson Heights,Downsview North",21,21,21,21,21,21
"Bathurst Quay,South Niagara,King and Spadina,Island airport,CN Tower,Harbourfront West,Railway Lands",18,18,18,18,18,18
Bayview Village,4,4,4,4,4,4
"Bedford Park,Lawrence Manor East",25,25,25,25,25,25
Berczy Park,55,55,55,55,55,55
"Birch Cliff,Cliffside West",4,4,4,4,4,4
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16


In [57]:
# One hot encoding: : Categorical variables are converted into a form (1 or 0) that could be provided to ML algorithms to do a better job in prediction

In [61]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="") # Convert categorical variable into dummy/indicator variables
toronto_onehot.insert(loc=0, column='Neighbourhood', value=toronto_venues['Neighbourhood'] )
toronto_onehot.shape

(2191, 270)

In [68]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood,Long Branch",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor,Wilson Heights,Downsview North",0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"Bathurst Quay,South Niagara,King and Spadina,I...",0.0,0.0,0.055556,0.055556,0.055556,0.111111,0.166667,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Bayview Village,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:] # take column values except [0]-> Neighborhood value which is not an integer
    row_categories_sorted = row_categories.sort_values(ascending=False) # from more to less -> descending manner
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [78]:
import numpy as np

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1)) # automatically it will pass to except condition when it couldn't find out indicators[3]

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood,Long Branch",Pizza Place,Skating Rink,Dance Studio,Sandwich Place,Athletics & Sports,Pub,Pool,Coffee Shop,Gym,Pharmacy
2,"Bathurst Manor,Wilson Heights,Downsview North",Coffee Shop,Bank,Sandwich Place,Supermarket,Restaurant,Deli / Bodega,Diner,Ice Cream Shop,Fried Chicken Joint,Frozen Yogurt Shop
3,"Bathurst Quay,South Niagara,King and Spadina,I...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Rental Car Location,Coffee Shop,Plane,Boat or Ferry,Sculpture Garden
4,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [85]:
from sklearn.cluster import KMeans
from sklearn.datasets.samples_generator import make_blobs

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', axis=1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 3, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 4, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [92]:
neighbourhoods_venues_sorted = neighbourhoods_venues_sorted.drop('Cluster Labels', axis=1)
neighbourhoods_venues_sorted

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood,Long Branch",Pizza Place,Skating Rink,Dance Studio,Sandwich Place,Athletics & Sports,Pub,Pool,Coffee Shop,Gym,Pharmacy
2,"Bathurst Manor,Wilson Heights,Downsview North",Coffee Shop,Bank,Sandwich Place,Supermarket,Restaurant,Deli / Bodega,Diner,Ice Cream Shop,Fried Chicken Joint,Frozen Yogurt Shop
3,"Bathurst Quay,South Niagara,King and Spadina,I...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Rental Car Location,Coffee Shop,Plane,Boat or Ferry,Sculpture Garden
4,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
5,"Bedford Park,Lawrence Manor East",Italian Restaurant,Sandwich Place,Restaurant,Coffee Shop,Comfort Food Restaurant,Butcher,Café,Pub,Pizza Place,Pharmacy
6,Berczy Park,Coffee Shop,Bakery,Beer Bar,Cocktail Bar,Cheese Shop,Café,Farmers Market,Seafood Restaurant,Restaurant,Breakfast Spot
7,"Birch Cliff,Cliffside West",College Stadium,Skating Rink,General Entertainment,Café,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
8,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Garden Center,Brewery,Park,Spa,Farmers Market,Light Rail Station,Fast Food Restaurant,Burrito Place,Restaurant
9,"CFB Toronto,Downsview East",Airport,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [93]:
# add clustering labels
neighbourhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_) # insert into [1] = 2nd column
neighbourhoods_venues_sorted

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,0,Lounge,Skating Rink,Latin American Restaurant,Clothing Store,Breakfast Spot,Yoga Studio,Dumpling Restaurant,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood,Long Branch",0,Pizza Place,Skating Rink,Dance Studio,Sandwich Place,Athletics & Sports,Pub,Pool,Coffee Shop,Gym,Pharmacy
2,"Bathurst Manor,Wilson Heights,Downsview North",0,Coffee Shop,Bank,Sandwich Place,Supermarket,Restaurant,Deli / Bodega,Diner,Ice Cream Shop,Fried Chicken Joint,Frozen Yogurt Shop
3,"Bathurst Quay,South Niagara,King and Spadina,I...",0,Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Boutique,Rental Car Location,Coffee Shop,Plane,Boat or Ferry,Sculpture Garden
4,Bayview Village,0,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore
5,"Bedford Park,Lawrence Manor East",0,Italian Restaurant,Sandwich Place,Restaurant,Coffee Shop,Comfort Food Restaurant,Butcher,Café,Pub,Pizza Place,Pharmacy
6,Berczy Park,0,Coffee Shop,Bakery,Beer Bar,Cocktail Bar,Cheese Shop,Café,Farmers Market,Seafood Restaurant,Restaurant,Breakfast Spot
7,"Birch Cliff,Cliffside West",0,College Stadium,Skating Rink,General Entertainment,Café,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
8,Business Reply Mail Processing Centre 969 Eastern,0,Yoga Studio,Garden Center,Brewery,Park,Spa,Farmers Market,Light Rail Station,Fast Food Restaurant,Burrito Place,Restaurant
9,"CFB Toronto,Downsview East",1,Airport,Park,Yoga Studio,Dumpling Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore


In [96]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = pd.merge(df, neighborhoods_venues_sorted, on='Neighbourhood')
toronto_merged.insert(3, 'Cluster Labels', kmeans.labels_)
toronto_merged.head()

,Postalcode,Borough,Neighbourhood,Cluster Labels,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge,Malvern",0,43.806686,-79.194353,Fast Food Restaurant,Drugstore,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Yoga Studio,Dim Sum Restaurant
1,M1C,Scarborough,"Highland Creek,Port Union,Rouge Hill",0,43.784535,-79.160497,History Museum,Bar,Yoga Studio,Dumpling Restaurant,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
2,M1E,Scarborough,"West Hill,Guildwood,Morningside",0,43.763573,-79.188711,Spa,Electronics Store,Intersection,Rental Car Location,Bank,Mexican Restaurant,Medical Center,Eastern European Restaurant,Dumpling Restaurant,Discount Store
3,M1G,Scarborough,Woburn,0,43.770992,-79.216917,Coffee Shop,Korean Restaurant,Yoga Studio,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
4,M1H,Scarborough,Cedarbrae,0,43.773136,-79.239476,Hakka Restaurant,Bakery,Bank,Thai Restaurant,Athletics & Sports,Gas Station,Caribbean Restaurant,Fried Chicken Joint,Doner Restaurant,Dog Run


In [97]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 43.653963, -79.387207.


In [99]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters